# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('cleaned', engine)
#print (colum)
X = df["message"]
Y = df[df.columns[4:]]
print(Y)

       related  request  offer  aid_related  medical_help  medical_products  \
0            1        0      0            0             0                 0   
1            1        0      0            1             0                 0   
2            1        0      0            0             0                 0   
3            1        1      0            1             0                 1   
4            1        0      0            0             0                 0   
5            0        0      0            0             0                 0   
6            1        0      0            0             0                 0   
7            1        1      0            1             0                 0   
8            0        0      0            0             0                 0   
9            1        1      0            1             1                 1   
10           1        1      0            1             1                 1   
11           0        0      0            0         

In [7]:
Y.related.unique()

array([1, 0, 2])

In [8]:
Y.loc[Y['related'] == 2] = 1
Y.related.unique()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


array([1, 0])

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline =Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC())),
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.2, random_state = 22)
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...bility=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
import numpy as np
y_pred = pipeline.predict(X_test)

In [13]:
i=0
for col in y_test:
    print('Feature {}: {}'.format(i+1, col))
    print(classification_report(y_test[col], y_pred[:, i]))
    i = i + 1
accuracy = (y_pred == y_test.values).mean()
print('Model accuracy is {:.3f}'.format(accuracy))

Feature 1: related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      4899
          1       0.77      1.00      0.87     16074

avg / total       0.59      0.77      0.67     20973

Feature 2: request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90     17210
          1       0.00      0.00      0.00      3763

avg / total       0.67      0.82      0.74     20973

Feature 3: offer
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     20713
          1       0.00      0.00      0.00       260

avg / total       0.98      0.99      0.98     20973

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.73     12152
          1       0.00      0.00      0.00      8821

avg / total       0.34      0.58      0.43     20973

Feature 5: medical_help
             precision    recall  f1-s

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.97      1.00      0.99     20377
          1       0.00      0.00      0.00       596

avg / total       0.94      0.97      0.96     20973

Feature 35: other_weather
             precision    recall  f1-score   support

          0       0.94      1.00      0.97     19738
          1       0.00      0.00      0.00      1235

avg / total       0.89      0.94      0.91     20973

Feature 36: direct_report
             precision    recall  f1-score   support

          0       0.80      1.00      0.89     16759
          1       0.00      0.00      0.00      4214

avg / total       0.64      0.80      0.71     20973

Model accuracy is 0.919


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
'''
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }
'''
parameters= [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test.values.argmax(axis=1), y_pred.argmax(axis=1))
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
display_results(cv, y_test, y_pred)

Labels: [0 1]
Confusion Matrix:
 [[20973]]
Accuracy: related                   0.766414
request                   0.820579
offer                     0.987603
aid_related               0.579412
medical_help              0.912745
medical_products          0.942259
search_and_rescue         0.964478
security                  0.974443
military                  0.959853
child_alone               0.992276
water                     0.928861
food                      0.880561
shelter                   0.904306
clothing                  0.976017
money                     0.969437
missing_people            0.980880
refugees                  0.959090
death                     0.946789
other_aid                 0.862156
infrastructure_related    0.927144
transport                 0.946407
buildings                 0.942497
electricity               0.971630
tools                     0.985934
hospitals                 0.980737
shops                     0.987746
aid_centers               0.980403
ot

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline1 =Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

print(sorted(pipeline1.get_params().keys()))
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.3)
pipeline1.fit(X_train, y_train)

['clf', 'clf__estimator', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__warm_start', 'clf__n_jobs', 'memory', 'steps', 'tfidf', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'vect', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 've

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [17]:
y_pred1 = pipeline1.predict(X_test)

In [18]:
i=0
for col in y_test:
    print('Feature {}: {}'.format(i+1, col))
    print(classification_report(y_test[col], y_pred1[:, i]))
    i = i + 1
accuracy = (y_pred1 == y_test.values).mean()
print('Model accuracy is {:.3f}'.format(accuracy))

Feature 1: related
             precision    recall  f1-score   support

          0       0.60      0.40      0.48      4304
          1       0.83      0.92      0.87     14048

avg / total       0.78      0.80      0.78     18352

Feature 2: request
             precision    recall  f1-score   support

          0       0.88      0.96      0.92     15079
          1       0.70      0.41      0.52      3273

avg / total       0.85      0.86      0.85     18352

Feature 3: offer
             precision    recall  f1-score   support

          0       0.99      0.99      0.99     18137
          1       0.17      0.16      0.17       215

avg / total       0.98      0.98      0.98     18352

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.74      0.82      0.77     10598
          1       0.71      0.60      0.65      7754

avg / total       0.72      0.73      0.72     18352

Feature 5: medical_help
             precision    recall  f1-s

In [19]:
param_grid = {
    'n_estimators': [5, 10, 50, 100, 200],
    'max_features': ['auto', 'log2', 'sqrt'],
    'bootstrap': [False, True],
    'max_depth':[3,5, 10, 20]}

'''param_grid = {"clf__estimator__max_depth": [3, 5, 10, 20],
                  'clf__estimator__n_estimators': [5, 10, 50, 100, 200],
              "clf__estimator__max_features": ['auto', 'log2', 'sqrt'],
              "clf__estimator__bootstrap": [True, False],
              "clf__estimator__criterion": ["gini", "entropy"]}
              '''
cv1 = GridSearchCV(pipeline1, param_grid=param_grid)

In [20]:
def display_results(cv1, y_test, y_pred1):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test.values.argmax(axis=1), y_pred1.argmax(axis=1))
    accuracy = (y_pred1 == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
display_results(cv1, y_test, y_pred1)

Labels: [0 1]
Confusion Matrix:
 [[17837    84    13   188     8     2     3     2    23     3    15     4
      3    10     5     3    12     3     2     1     1     2    70     9
      1    15     1     1    31]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0 

### 9. Export your model as a pickle file

In [21]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv1, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.